<a href="https://colab.research.google.com/github/PatuanPurba/JSC270_HW2_2025_PPurba/blob/main/Assignment_2_Part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [31]:
income94 = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)
income94.columns = ['age','workclass', 'fnlwgt','education','education_num','marital_status','occupation',
                    'relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','gross_income_group']

income94 = income94.applymap(lambda x: x.strip() if isinstance(x, str) else x) # Strip the unnecessary white space
income94.replace('?', np.nan, inplace=True)
income94.dropna(inplace=True)

<ipython-input-31-6d63cba0152a>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  income94 = income94.applymap(lambda x: x.strip() if isinstance(x, str) else x) # Strip the unnecessary white space


In [39]:
dependent_variable = 'hours_per_week'
independent_variables = ['age', 'workclass', 'education_num', 'sex', 'marital_status', 'gross_income_group', 'occupation']

In [40]:
def one_hot_nn(df, input_vector):
    for value in input_vector.unique()[1:]:
        df[value] = ((input_vector == value).to_numpy(dtype='float32'))

    return input_vector.unique()[0]


def get_exogenous(df_input, independent_variables):
    df = pd.DataFrame()
    default_dict = {}

    for value in independent_variables:
      if not (np.issubdtype(df_input[value].dtype, np.number)):
        default_value = one_hot_nn(df, df_input.loc[:, value])
        default_dict[value] = default_value
      else:
        df[value] = df_input.loc[:, value]

    result = df.replace({True: 1, False: 0})
    return result.infer_objects(copy=False), default_dict

In [41]:
df, default_dict = get_exogenous(income94, independent_variables)
default_dict

{'workclass': 'State-gov',
 'sex': 'Male',
 'marital_status': 'Never-married',
 'gross_income_group': '<=50K',
 'occupation': 'Adm-clerical'}

In [42]:
df

,age,Self-emp-not-inc,Private,Federal-gov,Local-gov,Self-emp-inc,Without-pay,education_num,Female,Married-civ-spouse,...,Other-service,Sales,Transport-moving,Farming-fishing,Machine-op-inspct,Tech-support,Craft-repair,Protective-serv,Armed-Forces,Priv-house-serv
0,39,0.0,0.0,0.0,0.0,0.0,0.0,13,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50,1.0,0.0,0.0,0.0,0.0,0.0,13,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38,0.0,1.0,0.0,0.0,0.0,0.0,9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,53,0.0,1.0,0.0,0.0,0.0,0.0,7,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,0.0,1.0,0.0,0.0,0.0,0.0,13,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,0.0,1.0,0.0,0.0,0.0,0.0,12,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32557,40,0.0,1.0,0.0,0.0,0.0,0.0,9,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
32558,58,0.0,1.0,0.0,0.0,0.0,0.0,9,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32559,22,0.0,1.0,0.0,0.0,0.0,0.0,9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
def backward_elimination(y, df_input, threshold):
  features = df_input.columns

  iteration = 0

  while True:
    iteration += 1
    model = sm.OLS(y, sm.add_constant(df_input)).fit()

    if iteration % 3 == 0:
      print(f'Iteration: {iteration}')
      print(model.summary())
      print("\n\n")

    p_values = model.pvalues
    max_p_value = p_values.max()

    if max_p_value > threshold:
      max_p_value_index = p_values.idxmax()
      df_input = df_input.drop(max_p_value_index, axis=1)
      print(f'On {iteration} iteration, dropped {max_p_value_index}')

    else:
      break

    print("\n\n\n\n")

  return model, df_input

In [44]:
model, df = backward_elimination(income94.loc[:, dependent_variable], df, threshold=0.05)

On 1 iteration, dropped Armed-Forces





On 2 iteration, dropped Tech-support





Iteration: 3
                            OLS Regression Results                            
Dep. Variable:         hours_per_week   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     206.5
Date:                Wed, 19 Feb 2025   Prob (F-statistic):               0.00
Time:                        00:36:20   Log-Likelihood:            -1.1514e+05
No. Observations:               30162   AIC:                         2.303e+05
Df Residuals:                   30134   BIC:                         2.306e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

In [49]:
print(model.summary())
print("\n\n\n")
print(default_dict)

                            OLS Regression Results                            
Dep. Variable:         hours_per_week   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     232.1
Date:                Wed, 19 Feb 2025   Prob (F-statistic):               0.00
Time:                        01:06:01   Log-Likelihood:            -1.1514e+05
No. Observations:               30162   AIC:                         2.303e+05
Df Residuals:                   30137   BIC:                         2.305e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    31.58

In [47]:
RMSE =  np.sqrt(np.mean((income94['hours_per_week'] - model.predict()) ** 2))
print(RMSE)

11.005712301111565
